# import

In [1]:
import pandas as pd
import json
import requests
from pathlib import Path
import numpy as np
import sys
import requests
import json
from matplotlib import pyplot as plt
import seaborn as sns
sns.set_context('talk')
%matplotlib inline


In [2]:
if sys.platform =='linux':
    path_d = Path('/mnt/d')
else:
    path_d = Path('D://')
data_dir = path_d/'data'/'dpe_full'   



# read exemple

In [3]:
td006 =pd.read_csv(data_dir/'full-db'/'csv'/'td006_batiment.csv',dtype=str)


In [92]:
dept = '38'
td008 =pd.read_csv(data_dir/dept/'td008_baie.csv',dtype=str)
td007 =pd.read_csv(data_dir/dept/'td007_paroi_opaque.csv',dtype=str)
td006 =pd.read_csv(data_dir/dept/'td006_batiment.csv',dtype=str)
td001 =pd.read_csv(data_dir/dept/'td001_dpe.csv',dtype=str)



In [93]:
from assets_orm import DPEMetaData

In [94]:
meta = DPEMetaData()
self=meta

In [95]:
from td008_processing import postprocessing_td008,merge_td008_tr_tv
from td007_processing import merge_td007_tr_tv,postprocessing_td007

In [96]:
td008 = merge_td008_tr_tv(td008)
td008 = postprocessing_td008(td008)
td007 = merge_td007_tr_tv(td007)
td007 = postprocessing_td007(td007)


In [97]:
td007 = td007.rename(columns={"id": "td007_paroi_opaque_id"})
td006 = td006.rename(columns={"id": "td006_batiment_id"})
td001 = td001.rename(columns={"id": "td001_dpe_id"})
td007 = td007.merge(td006[['td006_batiment_id','td001_dpe_id']],on='td006_batiment_id',how='left')

td008 = td008.merge(td007[['td007_paroi_opaque_id','td001_dpe_id']],on='td007_paroi_opaque_id',how='left')

# quantitatifs par paroi

In [248]:
# calcul des surfaces parois_opaque + paroi vitrée

surf = td008.groupby('td007_paroi_opaque_id')[['surface', 'surfacexnb_baie_calc','nb_baie_calc']].sum()
surf.columns = ['surface_baie_sum','surfacexnb_baie_calc_sum','nb_baie_calc']
surf['surfacexnb_baie_calc_sum']=surf.max(axis=1)

td007_m = td007.merge(surf,on ='td007_paroi_opaque_id',how='left')
td007_m['surface_paroi_opaque_calc']= td007_m.deperdition_thermique/(td007_m.coefficient_transmission_thermique_paroi.astype(float)*td007_m.tv001_valeur.astype(float))

td007_m['surface_paroi_totale_calc_v1']=td007_m.surface_paroi_opaque_calc+td007_m.surfacexnb_baie_calc_sum
td007_m['surface_paroi_totale_calc_v2']=td007_m.surface_paroi_opaque_calc+td007_m.surface_baie_sum

In [249]:
def calc_surf_approx_equality(s1,s2,rtol=0.05,atol=2):
    
    is_close = np.isclose(s1,s2,rtol=rtol)
    is_close = is_close|np.isclose(s1,s2,atol=atol)
    return is_close

In [250]:
is_surface_totale_v1=calc_surf_approx_equality(td007_m.surface_paroi,td007_m.surface_paroi_totale_calc_v1)
is_surface_totale_v2=calc_surf_approx_equality(td007_m.surface_paroi,td007_m.surface_paroi_totale_calc_v2)
is_surface_paroi_opaque=calc_surf_approx_equality(td007_m.surface_paroi,td007_m.surface_paroi_opaque_calc)
is_surface_paroi_opaque_deg=calc_surf_approx_equality(td007_m.surface_paroi,td007_m.surface_paroi_opaque_calc,rtol=0.1)


In [251]:
td007_m['qualif_surf']='NONDEF'
td007_m.loc[is_surface_paroi_opaque_deg,'qualif_surf']='surface_paroi=surface_paroi_opaque'
td007_m.loc[is_surface_totale_v1,'qualif_surf']='surface_paroi=surface_paroi_opaque+somme(surface baiesxnb_baies) v1'
td007_m.loc[is_surface_totale_v2,'qualif_surf']='surface_paroi=surface_paroi_opaque+somme(surface baies) v2'
td007_m.loc[is_surface_paroi_opaque,'qualif_surf']='surface_paroi=surface_paroi_opaque'
td007_m.loc[is_surface_paroi_opaque,'qualif_surf']='surface_paroi=surface_paroi_opaque'
td007_m.qualif_surf = td007_m.qualif_surf.astype('category')

In [102]:
td007_m['surface_paroi_totale_calc_v1']=td007_m.surface_paroi_opaque_calc+td007_m.surfacexnb_baie_calc_sum
td007_m['surface_paroi_totale_calc_v2']=td007_m.surface_paroi_opaque_calc+td007_m.surface_baie_sum

In [152]:
# infer surface paroi opaque

td007_m['surface_paroi_opaque_infer']=np.nan

is_surface_seul=td007_m.qualif_surf=='surface_paroi=surface_paroi_opaque'
td007_m.loc[is_surface_seul,'surface_paroi_opaque_infer']=td007_m.surface_paroi

is_surface_sum_baie_opaque_v1=td007_m.qualif_surf=='surface_paroi=surface_paroi_opaque+somme(surface baiesxnb_baies) v1'
td007_m.loc[is_surface_sum_baie_opaque_v1,'surface_paroi_opaque_infer']=td007_m.surface_paroi-td007_m.surfacexnb_baie_calc_sum

is_surface_sum_baie_opaque_v2=td007_m.qualif_surf=='surface_paroi=surface_paroi_opaque+somme(surface baies) v2'

td007_m.loc[is_surface_sum_baie_opaque_v2,'surface_paroi_opaque_infer']=td007_m.surface_paroi-td007_m.surface_baie_sum


null = td007_m.surface_paroi_opaque_infer.isnull()
td007_m.loc[null,'surface_paroi_opaque_infer']=td007_m.loc[null,'surface_paroi']

td007_m['surface_paroi_opaque_deperditive_infer']=td007_m.surface_paroi_opaque_infer

is_not_deper=(td007_m.deperdition_thermique==0)|(td007_m.tv001_valeur==0)
td007_m.loc[is_not_deper,'surface_paroi_opaque_deperditive_infer']=np.nan


td007_m['b_infer']=td007_m.deperdition_thermique/(td007_m.surface_paroi*td007_m.coefficient_transmission_thermique_paroi)
td007_m['surface_paroi_opaque_exterieur_infer']=td007_m.surface_paroi_opaque_infer
is_ext=td007_m.tv001_coefficient_reduction_deperditions_id=='TV001_001'
is_tv002=td007_m.tv002_local_non_chauffe_id.isnull()==False
is_tv001_non_ext=td007_m.tv001_coefficient_reduction_deperditions_id!='TV001_001'
is_non_ext_from_b_infer=td007_m.b_infer.round(2)<0.96
is_non_ext = (is_tv002) |(is_tv001_non_ext) |(is_non_ext_from_b_infer)
td007_m.loc[is_non_ext,'surface_paroi_opaque_exterieur_infer']=np.nan



In [104]:
# # concat orientation
# orientation=td008.groupby('td007_paroi_opaque_id')['orientation_infer'].apply(lambda x:'+'.join(list(set(x.tolist()))))
# td007_m = td007_m.merge(orientation,on='td007_paroi_opaque_id',how='left')

# # surface vitrée/ portes
# pivot = td008.pivot_table(index = 'td007_paroi_opaque_id', columns = 'cat_baie_simple_infer',
#                          values = 'surface')
# pivot.columns = pivot.columns.tolist()
# pivot['surface_vitree']=pivot[['baie vitrée','paroi en brique de verre ou polycarbonate']].sum(axis=1)

# td007_m=td007_m.merge(pivot.rename(columns={'porte':'surface_porte'})[['surface_porte','surface_vitree']]
#                       ,on='td007_paroi_opaque_id',how='left')

# pivot = td008.pivot_table(index = 'td007_paroi_opaque_id', columns = 'cat_baie_simple_infer',
#                          values = 'surfacexnb_baie_calc')
# pivot.columns = pivot.columns.tolist()
# pivot['surfacexnb_baie_calc_vitree']=pivot[['baie vitrée','paroi en brique de verre ou polycarbonate']].sum(axis=1)

# td007_m=td007_m.merge(pivot.rename(columns={'porte':'surfacexnb_baie_calc_porte'})[['surfacexnb_baie_calc_porte','surfacexnb_baie_calc_vitree']]
#                       ,on='td007_paroi_opaque_id',how='left')

In [105]:
# AGG orientation
td008_porte = td008.loc[td008.cat_baie_simple_infer=='porte']

td008_vitree = td008.loc[td008.cat_baie_simple_infer.isin(['baie vitrée',
                                                   
                                            'paroi en brique de verre ou polycarbonate'])]
orientation_agg=td008_vitree.groupby('td001_dpe_id')['orientation_infer'].apply(lambda x:','.join(list(set(x.tolist()))))


est_double=orientation_agg.str.count('Est')>1
ouest_double =orientation_agg.str.count('Ouest')>1
est_and_ouest_double = (est_double&ouest_double)

orientation_agg.loc[est_and_ouest_double]=orientation_agg.loc[est_and_ouest_double].str.replace('Est ou Ouest,','').str.replace(',Est ou Ouest','')

# AGG top freq type vitrage

from utils import agg_pond_top_freq,agg_pond_avg

td008_vitree['max_surface']=td008_vitree[['surfacexnb_baie_calc','surface']].max(axis=1)
type_vitrage_agg=agg_pond_top_freq(td008_vitree,'type_vitrage_simple_infer','max_surface','td001_dpe_id')

# AGG Ujn avg

td008_sel = td008_vitree.loc[td008_vitree.coefficient_transmission_thermique_baie>0]
Ujn_avg = agg_pond_avg(td008_sel,'coefficient_transmission_thermique_baie','max_surface','td001_dpe_id')

# SURFACES
td008_vitree = td008.loc[td008.cat_baie_simple_infer=='baie vitrée']

surf_vitree=td008_vitree.groupby('td001_dpe_id')['surfacexnb_baie_calc'].sum()
surf_porte=td008_porte.groupby('td001_dpe_id')['surfacexnb_baie_calc'].sum()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Cstbdata\MONWORKSPACE\Gitlab\de-dpe-processing\data_processing_cstb\traitement_donnees_metier\utils.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table[pond_col] = table[pond]


In [279]:
orientation_agg.value_counts()

Sud,Nord                                 13173
Nord,Sud,Ouest,Est                        8148
Ouest,Est                                 7870
Sud,Ouest,Est                             6785
Ouest,Sud,Nord,Est                        6177
                                         ...  
Ouest,Horizontale,Sud,Est,NONDEF,Nord        1
Horizontale,Nord,Ouest                       1
Horizontale,Sud,Ouest                        1
Est,NONDEF,Nord,Ouest                        1
Horizontale,Nord                             1
Name: orientation_infer, Length: 66, dtype: int64

In [201]:
td008_porte = td008.loc[td008.cat_baie_simple_infer=='porte']

td008_vitree = td008.loc[td008.cat_baie_simple_infer.isin(['baie vitrée',
                                                   
                                            'paroi en brique de verre ou polycarbonate'])]
surf_vitree=td008_vitree.groupby('td001_dpe_id')['surfacexnb_baie_calc'].sum()
surf_vitree.name = 'surface_vitree_totale'
surf_porte=td008_porte.groupby('td001_dpe_id')['surfacexnb_baie_calc'].sum()
surf_porte.name = 'surface_porte_totale'

In [238]:
surf_vitree_orient=td008_vitree.pivot_table(index='td001_dpe_id',columns='orientation_infer',values='surfacexnb_baie_calc',aggfunc='sum')

In [239]:
surf_vitree_orient.columns=[f'surfaces_vitree_orientee_{el.lower().replace(" ","_")}' for el in surf_vitree_orient.columns]

In [241]:
td007_murs = td007_m.loc[td007_m.tr014_type_parois_opaque_id.isin(['TR014_002','TR014_001'])]
td007_pb = td007_m.loc[td007_m.tr014_type_parois_opaque_id=='TR014_003']
td007_ph = td007_m.loc[td007_m.tr014_type_parois_opaque_id=='TR014_004']

In [243]:
surf_mur=td007_murs.groupby('td001_dpe_id')[['surface_paroi_opaque_infer',
                                             'surface_paroi_opaque_deperditive_infer','surface_paroi_opaque_exterieur_infer']].sum()
surf_pb=td007_pb.groupby('td001_dpe_id')[['surface_paroi_opaque_infer',
                                             'surface_paroi_opaque_deperditive_infer']].sum()

surf_ph=td007_pb.groupby('td001_dpe_id')[['surface_paroi_opaque_infer',
                                             'surface_paroi_opaque_deperditive_infer']].sum()

In [244]:
surf_mur.columns =['surf_murs_totale','surf_murs_deper','surf_murs_ext']
surf_pb.columns =['surf_pb_totale','surf_pb_deper']
surf_ph.columns =['surf_ph_totale','surf_ph_deper']


In [245]:
quantitatif = pd.concat([surf_mur,surf_pb,surf_ph,surf_vitree,surf_porte,surf_vitree_orient],axis=1)
quantitatif[quantitatif<0]=np.nan

In [265]:
quantitatif['ratio_surface_vitree']=quantitatif.surface_vitree_totale/quantitatif.surf_murs_ext
is_surf_ext = quantitatif.surf_murs_ext==0
ratio_surface_vitree = quantitatif['ratio_surface_vitree']
anomaly_ratio_surface_vitree = ratio_surface_vitree>0.95
quantitatif.loc[anomaly_ratio_surface_vitree,'ratio_surface_vitree']=np.nan

In [273]:
quantitatif

,surf_murs_totale,surf_murs_deper,surf_murs_ext,surf_pb_totale,surf_pb_deper,surf_ph_totale,surf_ph_deper,surface_vitree_totale,surface_porte_totale,surfaces_vitree_orientee_est,surfaces_vitree_orientee_est_ou_ouest,surfaces_vitree_orientee_horizontale,surfaces_vitree_orientee_nondef,surfaces_vitree_orientee_nord,surfaces_vitree_orientee_ouest,surfaces_vitree_orientee_sud,ratio_surface_vitree
1000252,146.22,146.22,146.22,115.03,115.03,115.03,115.03,19.22,3.50,3.24,NaN,NaN,NaN,1.14,4.65,10.19,0.131446
10003,116.83,116.83,85.73,141.00,141.00,141.00,141.00,27.47,NaN,7.16,NaN,NaN,NaN,16.29,NaN,4.02,0.320425
1000437,99.70,21.10,21.10,21.89,0.00,21.89,0.00,6.30,NaN,5.20,NaN,NaN,NaN,NaN,1.10,NaN,0.298578
1000817,97.75,97.75,0.00,88.03,88.03,88.03,88.03,6.96,NaN,NaN,NaN,NaN,6.96,NaN,NaN,NaN,NaN
1000866,196.33,196.33,151.53,65.00,65.00,65.00,65.00,20.30,NaN,NaN,NaN,NaN,NaN,2.47,4.07,13.76,0.133967
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99972,174.80,174.80,117.30,85.00,85.00,85.00,85.00,17.76,NaN,1.62,NaN,NaN,NaN,0.84,4.23,11.07,0.151407
999768,176.44,176.44,176.44,95.00,95.00,95.00,95.00,31.99,2.15,13.46,NaN,NaN,NaN,1.15,15.49,1.89,0.181308
999890,105.20,105.20,0.00,97.31,97.31,97.31,97.31,4.92,NaN,NaN,NaN,NaN,4.92,NaN,NaN,NaN,NaN
1312774,NaN,NaN,NaN,0.00,0.00,0.00,0.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [272]:
td001.nom_methode_dpe.value_counts()

Méthode 3CL                         35331
Méthode Facture                     35199
3CL - DPE                           30858
FACTURE - DPE                       22326
TROIS_CL_DPE                         9251
3CL-DPE                              7381
facture                              6666
FACTURE                              6534
3CL                                  3565
TH_C_E                               2286
th-CE                                1848
Factures                             1751
Th-BCE                               1723
FACTURE SEULE                        1675
TH_BCE                               1451
Facture                               963
DPE VIERGE                            839
MIXTE FACTURE+3CL - DPE pour ECS      404
Th-C-E                                163
FACTURES                                8
Th-C-E - DPE                            8
6.2 - DPE                               4
3cl                                     2
Name: nom_methode_dpe, dtype: int6

In [210]:
quantitatif.surf_murs_ext

count    81827.000000
mean       179.620439
std        491.647759
min        -33.580000
25%         16.000000
50%         46.380000
75%        102.310000
max       9327.090000
Name: surf_murs_ext, dtype: float64

In [209]:
quantitatif.surface_vitree_totale.describe()

count    80176.000000
mean        63.443135
std        232.311838
min          0.000000
25%          8.450000
50%         13.330000
75%         21.640000
max       7074.270000
Name: surface_vitree_totale, dtype: float64

In [208]:
quantitatif['ratio_surface_vitree'].describe()

count    7.989200e+04
mean              inf
std               NaN
min     -2.926471e+01
25%      1.748946e-01
50%      2.816167e-01
75%      5.576092e-01
max               inf
Name: ratio_surface_vitree, dtype: float64

In [181]:
td007_murs.loc[td007_murs.td001_dpe_id=="1000437"][['tv001_coefficient_reduction_deperditions_id',
                                                    'surface_paroi','surface_paroi_opaque_deperditive_infer','tv002_local_non_chauffe_id']]

,tv001_coefficient_reduction_deperditions_id,surface_paroi,surface_paroi_opaque_deperditive_infer,tv002_local_non_chauffe_id
50591,TV001_001,5.2,5.2,NaN
50592,TV001_100,17.6,NaN,NaN
50593,TV001_001,2.9,2.9,NaN
50594,TV001_100,21.6,NaN,NaN
50595,TV001_001,6.5,6.5,NaN
50596,TV001_100,19.7,NaN,NaN
50597,TV001_001,6.5,6.5,NaN
50598,TV001_100,19.7,NaN,NaN


In [164]:
surf_mur

,surface_paroi_opaque_infer,surface_paroi_opaque_deperditive_infer,surface_paroi_opaque_exterieur_infer
td001_dpe_id,,,
1000252,146.22,146.22,146.22
10003,116.83,116.83,85.73
1000437,99.70,21.10,21.10
1000866,196.33,196.33,151.53
1000917,142.51,142.51,124.99
...,...,...,...
999222,80.88,80.88,66.57
999298,82.30,82.30,63.60
99972,174.80,174.80,117.30


# quantitatifs batiment

In [62]:
td008.cat_baie_simple_infer.value_counts()

baie vitrée                                  475518
porte                                         51169
paroi en brique de verre ou polycarbonate       599
Name: cat_baie_simple_infer, dtype: int64

# quantitatifs dpe